<a href="https://colab.research.google.com/github/matheusgpantoja/LPO/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyomo

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.3 MB/s eta 0:00:00


In [ ]:
# Importing the modules

from pyomo.environ import *
import pandas as pd

#### Parameters

<!--

-->

In [ ]:
# ===========================================
#           Topology Information            #
# ===========================================

total_nodes = 3 # [0, 1, 2, 3]

# CPU (Core),

total_resources = 1
resource_capacity = {'CPU' : 8,}


# ===================================
#       Functions Information       #
# ===================================

# (0)NAT (1)FW (2)TM (3)WOC (4)VOC (5)IDS
# =======================================
total_functions = 5


# Set of Functions
# id = Function Identifier
# cf = Each function require X CPU core
# bf = Traffic Processin Capacity (Function) in Mb/s (200Mb/s = 200000Kb/s)

#            (id, cf, bf)
functions = [(0, 4, 10000),
             (1, 4, 10000),
             (2, 4, 10000),
             (3, 4, 10000),
             (4, 4, 10000),
             (5, 4, 10000)]


# =============================
#     Services Information    #
# =============================

# wb = Web Service, vip = VoIP, vs = Video Streaming, og = Online Game
#      ===========        ====       ===============       ===========


# [function requested, , Bandwidth (kbps)]

services = {'wb' : {    'function' : 1,
                        'bandwidth' : 100, },
# ----------------------------------------------
            'vip' : {   'function' : 0,
                        'bandwidth' : 64, },
# ----------------------------------------------
            'vs' : {    'function' : 4,
                        'bandwidth' : 100, },
# ----------------------------------------------
            'og' : {    'function' : 5,
                        'bandwidth' : 50, },
# ----------------------------------------------
            }


# =============================
#     Demands Information    #
# =============================

total_demandas = [200, 350, 500, 650, 800, 1000, 1250]



#### Funções

In [ ]:
def load_file(file):
    """
    """

    with open(file, 'rb') as file:
        return pickle.load(file)


In [ ]:
def load_file(file):
    """
    """

    with open(file, 'rb') as file:
        return pickle.load(file)


In [ ]:
def alocacao_demandas(model):
    """
    """


    global total_demands

    col1 = []
    for d in range(total_demands+1):
        col1.append(f"Demand {d}")

    col2 = []
    for s in range(pm.total_nodes+1):
        col2.append(f"Server {s}")

    allocated = pd.DataFrame(columns = col2, index = list(set(col1)))

    for d in range(total_demands+1):
        for s in range(pm.total_nodes+1):
            if (value(model.d[d, s])) == 1:
                allocated.loc[f"Demand {d}", f"Server {s}"] = 1
            else:
                allocated.loc[f"Demand {d}", f"Server {s}"] = '.'

    #print("A demanda X está sendo atendida no Servidor Y")

    return allocated

In [ ]:
def alocacao_funcoes(model):
    """
    """


    col1 = []
    for f in range(pm.total_functions+1):
        col1.append(f"Functions {f}")

    col2 = []
    for s in range(pm.total_nodes+1):
        col2.append(f"Server {s}")

    allocated_functions = pd.DataFrame(columns = col2, index = list(set(col1)))
    allocated_functions.sort_index(inplace = True)

    for f in range(pm.total_functions+1):
        for s in range(pm.total_nodes+1):
            if (value(model.f[f, s])) == 1:
                allocated_functions.loc[f"Functions {f}", f"Server {s}"] = 1
            else:
                allocated_functions.loc[f"Functions {f}", f"Server {s}"] = '.'

    #print("A Função X está sendo alocada no Servidor Y")
    return allocated_functions

#### MOdelo Load Distribution

In [ ]:
def alocacao_funcoes(model):
    """
    """


    col1 = []
    for f in range(pm.total_functions+1):
        col1.append(f"Functions {f}")

    col2 = []
    for s in range(pm.total_nodes+1):
        col2.append(f"Server {s}")

    allocated_functions = pd.DataFrame(columns = col2, index = list(set(col1)))
    allocated_functions.sort_index(inplace = True)

    for f in range(pm.total_functions+1):
        for s in range(pm.total_nodes+1):
            if (value(model.f[f, s])) == 1:
                allocated_functions.loc[f"Functions {f}", f"Server {s}"] = 1
            else:
                allocated_functions.loc[f"Functions {f}", f"Server {s}"] = '.'

    #print("A Função X está sendo alocada no Servidor Y")
    return allocated_functionsdef allocated_demands(model):
    """
    """
    allocated = sum( model.d[d, i]
                for d in model.Demands
                for i in model.Nodes
            )

    economy = sum ( model.f[f, i]
                    for f in model.Functions
                    for i in model.Nodes
                    )

    return (allocated) - (0.001 * economy)

In [ ]:
def constraint_demand_allocated(model, d):
    '''
    Each demand must be allocated on a maximum of one server.
    '''
    return sum( model.d[d, s] for s in model.Nodes) <= 1

In [ ]:
def constraint_demand_functions(model, d, s, f):
    '''
    A demand can only be allocated to a server that has the function it needs.
    '''
    function = demands[d]['service']['function']

    if function == f:
        return model.f[f, s] - model.d[d, s] >= 0
    else:
        return Constraint.Skip

In [ ]:
def constraint_server_capacity(model, s):
    '''
    A server must not exceed its processing/storage caabilities.
    '''
    return sum(model.f[f, s] * pm.functions[f][1] for f in model.Functions) <= pm.resource_capacity['CPU']


In [ ]:
def constraint_functions_capacity(model, s, f):
    '''
    A functions must not exceed its processing caabilities.
    '''

    return sum(model.d[d, s] * demands[d]['service']['bandwidth'] for d in model.Demands if (demands[d]['service']['function'] == f)) \
            <= pm.functions[f][2] * model.f[f, s]

In [ ]:
def model_creation(total_demands):

    """
    """

    # Creating the Model
    model = ConcreteModel()

    # Creating the Sets
    model.Nodes = RangeSet(0, pm.total_nodes) # Set of Nodes
    model.Demands = RangeSet(0, total_demands) # Set of Demands
    model.Functions = RangeSet(0, pm.total_functions) # Set od Funcitons

    # Creating Variables

    # allocation demand
    model.d = Var(  model.Demands,
                    model.Nodes,
                    within = Binary,
                    initialize = 0
                )

    # allocation function
    model.f = Var(  model.Functions,
                    model.Nodes,
                    within = Binary,
                    initialize = 0
                )


    # Objective Function : Maximize the number of demands allocated
    model.objective = Objective(    rule = allocated_demands,
                                    sense = maximize
                                )

    # Constraint 01
    model.C1 = Constraint(  model.Demands,
                            rule = constraint_demand_allocated
                        )

    # Constraint 02
    model.C2 = Constraint(  model.Demands,
                            model.Nodes,
                            model.Functions,
                            rule = constraint_demand_functions
                        )

    # Constraint 03
    model.C3 = Constraint(  model.Nodes,
                            rule = constraint_server_capacity
                        )

    # Constraint 04
    model.C4 = Constraint(  model.Nodes,
                            model.Functions,
                            rule = constraint_functions_capacity
                        )

    return model

#### Execução

In [ ]:
for x in pm.total_demandas:
    resultados = []

    for d in range(100):

        file = f"demandas//{x}_demanda_{d}.pkl"

        # Set of demands
        # Demand Identifier, Source, Service, Allocated
        demands = load_file(file)
        total_demands = len(demands) - 1 # Total de n - 1 demandas

        model = model_creation(total_demands)

        # Selecting and Creating the Solver
        # ----------------------------
        solver = SolverFactory('cplex')

        # Calling the solver and solving the model
        # ----------------------------

        results = solver.solve( model,
                                #tee = True,
                                #keepfiles = True,
                            )


        fitness = value(model.objective)
        print(f"A função objetivo retornou o valor: {fitness}")
        print(f"A função objetivo REAL é: {sum(value(model.d[:,:]))}")

        allocated = alocacao_demandas(model)
        allocated_functions = alocacao_funcoes(model)
        resultados.append([fitness, allocated, allocated_functions])

    file = f'D:\\Programação - Códigos\\Alocação de Requisições - Funções de Rede Virtual\\Resultados\\pl_{x}'
    save_file(resultados, file)